# Module 00: AI Dynamo Architecture Overview

> **Systems Engineering Perspective**: Understanding Dynamo as a distributed systems problem

---

## Learning Objectives

By the end of this module, you will:

1. **Map** Dynamo components to familiar distributed systems patterns
2. **Understand** why LLM inference requires specialized infrastructure
3. **Trace** the lifecycle of a request through the system
4. **Identify** the role of each infrastructure component (etcd, NATS, NIXL)
5. **Explain** disaggregated serving using systems terminology

## Prerequisites Check

This module is conceptual - no Dynamo installation required yet. We just need basic Python to visualize concepts.

In [1]:
# Basic environment check
import sys
import platform

print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Architecture: {platform.machine()}")

# Check if we're on a GPU-enabled system (informational only)
try:
    import subprocess
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], 
                          capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        print(f"\nGPU(s) detected:")
        for i, line in enumerate(result.stdout.strip().split('\n')):
            print(f"  GPU {i}: {line}")
    else:
        print("\nNo GPU detected (that's OK for this module)")
except Exception as e:
    print(f"\nCould not check GPU: {e} (that's OK for this module)")

Python version: 3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]
Platform: Linux-6.11.0-1016-nvidia-aarch64-with-glibc2.39
Architecture: aarch64

GPU(s) detected:
  GPU 0: NVIDIA GB10, [N/A]


---

## Part 1: The Mental Model

### From Web Services to LLM Inference

If you've built distributed web services, you already understand 90% of Dynamo. Let's map the concepts:

| **Distributed Systems Concept** | **Dynamo Equivalent** | **Why It Matters** |
|--------------------------------|----------------------|-------------------|
| Reverse proxy (Nginx) | Frontend | HTTP handling, API translation |
| Load balancer with session affinity | Router | KV cache-aware request routing |
| Backend service instances | Workers | GPU inference engines |
| Service registry (Consul/ZK) | etcd | Dynamic worker discovery |
| Message queue (Kafka/RabbitMQ) | NATS JetStream | KV cache events, metrics |
| Session cache (Redis) | KV Cache | Attention state per request |
| DMA / RDMA | NIXL | Zero-copy GPU-to-GPU transfer |

In [2]:
# Let's visualize this mapping
component_mapping = {
    "Frontend": {
        "systems_analog": "Nginx reverse proxy",
        "language": "Rust",
        "responsibility": "HTTP handling, OpenAI API compatibility, request validation",
        "port": 8000
    },
    "Router": {
        "systems_analog": "Load balancer with session affinity",
        "language": "Rust",
        "responsibility": "Route requests to optimal worker based on KV cache hits",
        "port": "Internal"
    },
    "Worker": {
        "systems_analog": "Backend service instance",
        "language": "Python (vLLM/SGLang/TRT-LLM)",
        "responsibility": "GPU inference - model loading, prefill, decode",
        "port": 8080
    },
    "etcd": {
        "systems_analog": "Consul / ZooKeeper",
        "language": "Go",
        "responsibility": "Service discovery, worker registration, health tracking",
        "port": 2379
    },
    "NATS": {
        "systems_analog": "Kafka / RabbitMQ",
        "language": "Go",
        "responsibility": "Event streaming, KV cache notifications, metrics",
        "port": 4222
    },
    "NIXL": {
        "systems_analog": "RDMA / DMA",
        "language": "C/Rust",
        "responsibility": "Zero-copy KV cache transfer between GPUs",
        "port": "InfiniBand"
    }
}

print("Dynamo Component Reference")
print("=" * 80)
for component, details in component_mapping.items():
    print(f"\n{component}")
    print(f"  Systems Analog : {details['systems_analog']}")
    print(f"  Language       : {details['language']}")
    print(f"  Responsibility : {details['responsibility']}")
    print(f"  Default Port   : {details['port']}")

Dynamo Component Reference

Frontend
  Systems Analog : Nginx reverse proxy
  Language       : Rust
  Responsibility : HTTP handling, OpenAI API compatibility, request validation
  Default Port   : 8000

Router
  Systems Analog : Load balancer with session affinity
  Language       : Rust
  Responsibility : Route requests to optimal worker based on KV cache hits
  Default Port   : Internal

Worker
  Systems Analog : Backend service instance
  Language       : Python (vLLM/SGLang/TRT-LLM)
  Responsibility : GPU inference - model loading, prefill, decode
  Default Port   : 8080

etcd
  Systems Analog : Consul / ZooKeeper
  Language       : Go
  Responsibility : Service discovery, worker registration, health tracking
  Default Port   : 2379

NATS
  Systems Analog : Kafka / RabbitMQ
  Language       : Go
  Responsibility : Event streaming, KV cache notifications, metrics
  Default Port   : 4222

NIXL
  Systems Analog : RDMA / DMA
  Language       : C/Rust
  Responsibility : Zero-copy KV ca

---

## Part 2: Why LLM Inference is Different

### The Two-Phase Nature of LLM Requests

Unlike traditional web requests (receive → process → respond), LLM inference has **two distinct phases** with completely different resource profiles:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        LLM Request Lifecycle                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│   User Prompt ───▶ [PREFILL PHASE] ───▶ [DECODE PHASE] ───▶ Response   │
│                                                                         │
│   "Explain           Process entire       Generate tokens               │
│    quantum           prompt at once       one-by-one                    │
│    physics"          (compute KV cache)   (streaming)                   │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Let's understand the resource profiles of each phase

phase_characteristics = {
    "Prefill Phase": {
        "description": "Process entire input prompt, compute attention for all tokens",
        "compute_pattern": "Compute-bound (matrix multiplications)",
        "gpu_utilization": "HIGH (80-95%)",
        "memory_access": "Sequential, cache-friendly",
        "batching": "Highly parallelizable across prompts",
        "latency_metric": "Time-to-First-Token (TTFT)",
        "systems_analog": "Batch processing job (like video transcoding)"
    },
    "Decode Phase": {
        "description": "Generate output tokens one at a time, autoregressively",
        "compute_pattern": "Memory-bandwidth bound (reading KV cache)",
        "gpu_utilization": "LOW per token (20-40%)",
        "memory_access": "Random access to KV cache",
        "batching": "Limited by memory, not compute",
        "latency_metric": "Inter-Token Latency (ITL)",
        "systems_analog": "Real-time streaming (like live video)"
    }
}

print("LLM Inference Phase Comparison")
print("=" * 80)

for phase, chars in phase_characteristics.items():
    print(f"\n{phase}")
    print("-" * 40)
    for key, value in chars.items():
        print(f"  {key:20}: {value}")

### The Key Insight: Disaggregation

Running prefill and decode on the same GPU is like running your:
- **Batch ETL jobs** (CPU-intensive, high throughput)
- **Real-time API servers** (latency-sensitive, consistent response time)

...on the same machine. It works, but it's inefficient.

**Dynamo's solution**: Separate prefill workers from decode workers.

In [ ]:
# Visualizing monolithic vs disaggregated serving

monolithic_timeline = """
MONOLITHIC SERVING (Traditional)
================================

GPU 1: [====PREFILL====][==DECODE==][==DECODE==][==DECODE==]....
                        ↑
                        TTFT (Time to First Token)

GPU 2: [====PREFILL====][==DECODE==][==DECODE==][==DECODE==]....

Problem: 
- During PREFILL, GPU is compute-saturated (good!)
- During DECODE, GPU is memory-bandwidth limited (underutilized!)
- New requests must wait for current decode to finish
"""

disaggregated_timeline = """
DISAGGREGATED SERVING (Dynamo)
==============================

Prefill GPU 1: [==PREFILL==][==PREFILL==][==PREFILL==][==PREFILL==]...
                     │            │            │
                     │ KV Cache   │ KV Cache   │ KV Cache
                     ▼ (via NIXL) ▼            ▼
Decode GPU 2:  [=D=][=D=][=D=][=D=][=D=][=D=][=D=][=D=][=D=]...
Decode GPU 3:  [=D=][=D=][=D=][=D=][=D=][=D=][=D=][=D=][=D=]...

Benefits:
- Prefill GPU stays compute-saturated (optimal for throughput)
- Decode GPUs batch many concurrent decodes (better memory utilization)
- Independent scaling of prefill vs decode capacity
"""

print(monolithic_timeline)
print(disaggregated_timeline)

---

## Part 3: The KV Cache - Your New Best Friend

### What is the KV Cache?

In transformer models, the **attention mechanism** computes relationships between all tokens. For each layer, it produces:
- **K (Key)**: What information this token provides
- **V (Value)**: The actual information content

**The KV cache stores these computed attention states** so they don't need to be recomputed.

### Systems Analogy: Memoization / Session Cache

Think of it like:
- **Web sessions**: User logs in once, session token avoids re-authentication
- **Function memoization**: Expensive computation cached by input
- **Database query cache**: Same query returns cached results

In [ ]:
# Let's calculate KV cache sizes for common models

def calculate_kv_cache_size(
    num_layers: int,
    num_kv_heads: int,
    head_dim: int,
    sequence_length: int,
    dtype_bytes: int = 2  # FP16 = 2 bytes, FP8 = 1 byte
) -> dict:
    """
    Calculate KV cache memory requirements.
    
    KV Cache size = 2 * num_layers * num_kv_heads * head_dim * seq_len * dtype_bytes
    (2 for K and V)
    """
    bytes_per_token = 2 * num_layers * num_kv_heads * head_dim * dtype_bytes
    total_bytes = bytes_per_token * sequence_length
    
    return {
        "bytes_per_token": bytes_per_token,
        "mb_per_token": bytes_per_token / (1024 * 1024),
        "total_bytes": total_bytes,
        "total_mb": total_bytes / (1024 * 1024),
        "total_gb": total_bytes / (1024 * 1024 * 1024)
    }

# Common model configurations
models = {
    "TinyLlama-1.1B": {"num_layers": 22, "num_kv_heads": 4, "head_dim": 64},
    "Llama-3-8B": {"num_layers": 32, "num_kv_heads": 8, "head_dim": 128},
    "Llama-3-70B": {"num_layers": 80, "num_kv_heads": 8, "head_dim": 128},
    "DeepSeek-R1-671B": {"num_layers": 61, "num_kv_heads": 128, "head_dim": 128},  # MoE
}

# Calculate for different sequence lengths
seq_lengths = [512, 2048, 8192, 32768, 131072]  # Up to 128K context

print("KV Cache Memory Requirements (FP16)")
print("=" * 90)
print(f"{'Model':<20} {'Seq Len':>10} {'Per Token':>12} {'Total Cache':>15} {'% of H100 80GB':>15}")
print("-" * 90)

for model_name, config in models.items():
    for seq_len in seq_lengths:
        result = calculate_kv_cache_size(
            num_layers=config["num_layers"],
            num_kv_heads=config["num_kv_heads"],
            head_dim=config["head_dim"],
            sequence_length=seq_len
        )
        h100_pct = (result["total_gb"] / 80) * 100
        
        if seq_len == 512:  # Only print model name once
            print(f"{model_name:<20} {seq_len:>10} {result['mb_per_token']:>10.3f} MB {result['total_gb']:>12.2f} GB {h100_pct:>13.1f}%")
        else:
            print(f"{'':<20} {seq_len:>10} {result['mb_per_token']:>10.3f} MB {result['total_gb']:>12.2f} GB {h100_pct:>13.1f}%")
    print()

### Why KV Cache Routing Matters

Look at those numbers! For a 70B model with 32K context, the KV cache is **20+ GB**.

Now imagine a multi-turn conversation:

```
Turn 1: "Explain quantum physics"           → Compute 500 tokens of KV cache
Turn 2: "Now explain entanglement"          → Reuse 500 tokens, add 100 more
Turn 3: "What about quantum computing?"     → Reuse 600 tokens, add 100 more
```

**Without KV-aware routing**: Each turn might go to a different worker, recomputing from scratch.

**With KV-aware routing**: Dynamo routes to the worker that already has the prefix cached.

In [ ]:
# Simulating the impact of KV cache routing

import random

def simulate_routing(
    num_requests: int,
    num_workers: int,
    conversation_turns: int,
    tokens_per_turn: int,
    compute_time_per_token_ms: float,  # Time to compute KV cache for one token
    routing_strategy: str  # "random" or "kv_aware"
) -> dict:
    """
    Simulate routing efficiency for multi-turn conversations.
    """
    total_compute_time = 0
    total_cache_hits = 0
    total_cache_misses = 0
    
    # Track which worker has which conversation's cache
    conversation_worker = {}  # conversation_id -> worker_id
    worker_cache = {i: set() for i in range(num_workers)}  # worker_id -> set of conversation_ids
    
    for conv_id in range(num_requests):
        for turn in range(conversation_turns):
            prefix_tokens = turn * tokens_per_turn  # Tokens from previous turns
            new_tokens = tokens_per_turn
            
            if routing_strategy == "random":
                worker = random.randint(0, num_workers - 1)
            else:  # kv_aware
                # Route to worker that has this conversation's cache
                worker = conversation_worker.get(conv_id, random.randint(0, num_workers - 1))
            
            # Check cache hit
            if conv_id in worker_cache[worker]:
                # Cache hit - only compute new tokens
                compute_time = new_tokens * compute_time_per_token_ms
                total_cache_hits += 1
            else:
                # Cache miss - compute all tokens (prefix + new)
                compute_time = (prefix_tokens + new_tokens) * compute_time_per_token_ms
                total_cache_misses += 1
                # Update cache
                worker_cache[worker].add(conv_id)
            
            total_compute_time += compute_time
            conversation_worker[conv_id] = worker
    
    total_turns = num_requests * conversation_turns
    return {
        "total_compute_time_ms": total_compute_time,
        "avg_latency_ms": total_compute_time / total_turns,
        "cache_hit_rate": total_cache_hits / total_turns * 100,
        "cache_misses": total_cache_misses,
        "cache_hits": total_cache_hits
    }

# Run simulation
params = {
    "num_requests": 100,          # 100 conversations
    "num_workers": 4,             # 4 worker GPUs
    "conversation_turns": 5,      # 5 turns per conversation
    "tokens_per_turn": 500,       # 500 tokens per turn
    "compute_time_per_token_ms": 0.5  # 0.5ms per token prefill
}

print("KV Cache Routing Simulation")
print("=" * 60)
print(f"Scenario: {params['num_requests']} conversations, {params['conversation_turns']} turns each")
print(f"Workers: {params['num_workers']}, Tokens/turn: {params['tokens_per_turn']}")
print()

random_result = simulate_routing(**params, routing_strategy="random")
kv_aware_result = simulate_routing(**params, routing_strategy="kv_aware")

print(f"{'Metric':<25} {'Random Routing':>20} {'KV-Aware Routing':>20}")
print("-" * 65)
print(f"{'Total Compute Time':<25} {random_result['total_compute_time_ms']:>17.0f} ms {kv_aware_result['total_compute_time_ms']:>17.0f} ms")
print(f"{'Avg Latency per Turn':<25} {random_result['avg_latency_ms']:>17.1f} ms {kv_aware_result['avg_latency_ms']:>17.1f} ms")
print(f"{'Cache Hit Rate':<25} {random_result['cache_hit_rate']:>17.1f}% {kv_aware_result['cache_hit_rate']:>17.1f}%")
print(f"{'Cache Misses':<25} {random_result['cache_misses']:>20} {kv_aware_result['cache_misses']:>20}")
print()

speedup = random_result['total_compute_time_ms'] / kv_aware_result['total_compute_time_ms']
print(f"Speedup with KV-aware routing: {speedup:.1f}x")

---

## Part 4: Infrastructure Components

### The Control Plane: etcd + NATS

Dynamo uses two complementary systems for coordination:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        CONTROL PLANE                                    │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  etcd (Service Discovery)              NATS JetStream (Events)          │
│  ┌─────────────────────────┐          ┌─────────────────────────┐       │
│  │ /dynamo/workers/        │          │ dynamo.kv.events        │       │
│  │   worker-1: {endpoint,  │          │   • cache.hit           │       │
│  │              gpu,       │          │   • cache.miss          │       │
│  │              status}    │          │   • cache.evict         │       │
│  │   worker-2: {...}       │          │                         │       │
│  │                         │          │ dynamo.planner.metrics  │       │
│  │ Watch-based updates     │          │   • queue_depth         │       │
│  │ Lease-based TTL         │          │   • ttft_p95            │       │
│  └─────────────────────────┘          │   • gpu_utilization     │       │
│                                       └─────────────────────────┘       │
│                                                                         │
│  USE CASE: "Who exists?"              USE CASE: "What's happening?"     │
│  - Worker registration                - Real-time KV cache events        │
│  - Health checks                      - Metrics streaming                │
│  - Configuration                      - Planner decision triggers        │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Example: What worker registration looks like in etcd

import json
from datetime import datetime

# This is what a worker would register in etcd
worker_registration = {
    "worker_id": "worker-dgxspark01-gpu0",
    "endpoint": "10.0.0.1:8080",
    "grpc_endpoint": "10.0.0.1:8081",
    "gpu": {
        "name": "GH200",
        "memory_gb": 96,
        "compute_capability": "9.0"
    },
    "model": {
        "name": "meta-llama/Llama-3.1-8B-Instruct",
        "quantization": "FP16",
        "max_seq_len": 8192
    },
    "role": "prefill",  # or "decode" or "unified"
    "status": "healthy",
    "current_load": {
        "active_requests": 3,
        "queue_depth": 7,
        "gpu_memory_used_gb": 45.2
    },
    "registered_at": datetime.now().isoformat(),
    "lease_ttl_seconds": 30
}

print("Example: Worker Registration in etcd")
print("Key: /dynamo/workers/worker-dgxspark01-gpu0")
print("Value:")
print(json.dumps(worker_registration, indent=2))

print("\n" + "=" * 60)
print("\nExample etcdctl commands:")
print("-" * 60)
print("# Register a worker (with 30-second lease)")
print('etcdctl lease grant 30')
print('etcdctl put /dynamo/workers/worker-1 \'{"endpoint": "..."}\' --lease=<LEASE_ID>')
print()
print("# List all workers")
print('etcdctl get --prefix /dynamo/workers/')
print()
print("# Watch for changes (Frontend uses this)")
print('etcdctl watch --prefix /dynamo/workers/')

In [ ]:
# Example: What KV cache events look like in NATS

kv_cache_events = [
    {
        "subject": "dynamo.kv.cached",
        "data": {
            "worker_id": "worker-1",
            "prefix_hash": "a3f2b8c1d4e5",
            "prefix_tokens": 512,
            "blocks_allocated": 64,
            "memory_mb": 256,
            "timestamp": "2026-02-01T10:30:00Z"
        },
        "description": "Worker cached a new prefix - Router updates its radix tree"
    },
    {
        "subject": "dynamo.kv.hit",
        "data": {
            "worker_id": "worker-1",
            "prefix_hash": "a3f2b8c1d4e5",
            "hit_tokens": 512,
            "request_id": "req-12345",
            "saved_compute_ms": 256
        },
        "description": "Cache hit - request avoided recomputing 512 tokens"
    },
    {
        "subject": "dynamo.kv.evicted",
        "data": {
            "worker_id": "worker-1",
            "prefix_hash": "b4c3d2e1f0a9",
            "reason": "memory_pressure",
            "freed_mb": 512
        },
        "description": "Cache eviction - Router removes from radix tree"
    },
    {
        "subject": "dynamo.planner.metrics",
        "data": {
            "worker_id": "worker-1",
            "role": "prefill",
            "queue_depth": 15,
            "ttft_p50_ms": 120,
            "ttft_p95_ms": 450,
            "gpu_utilization": 0.92
        },
        "description": "Planner monitors this to decide scaling"
    }
]

print("Example: KV Cache Events in NATS JetStream")
print("=" * 70)

for event in kv_cache_events:
    print(f"\nSubject: {event['subject']}")
    print(f"Purpose: {event['description']}")
    print(f"Data: {json.dumps(event['data'], indent=2)}")
    print("-" * 70)

### The Data Plane: NIXL (RDMA for KV Cache)

When prefill completes, the KV cache needs to move to the decode worker. This is where NIXL shines.

```
TRADITIONAL TCP TRANSFER:
GPU A → PCIe → CPU → TCP Stack → NIC → Network → NIC → TCP → CPU → PCIe → GPU B
        ↑           ↑                              ↑           ↑
        Copy        Kernel                         Kernel      Copy
        
Latency: ~100-500μs for small transfers, scales poorly
CPU overhead: High (copies, syscalls)

─────────────────────────────────────────────────────────────────────────────

NIXL WITH RDMA (GPU-Direct):
GPU A ─────────────────── InfiniBand ─────────────────── GPU B
                    (zero-copy, kernel bypass)
                    
Latency: ~1-5μs
CPU overhead: Near zero
```

In [ ]:
# Comparing transfer times: TCP vs RDMA

def estimate_transfer_time(
    data_size_gb: float,
    method: str
) -> dict:
    """
    Estimate KV cache transfer time.
    """
    if method == "tcp_100gbe":
        # 100GbE = ~12.5 GB/s theoretical, ~10 GB/s practical
        bandwidth_gbps = 10
        latency_overhead_ms = 0.5  # TCP/IP stack overhead
        cpu_overhead = "High (multiple copies)"
    elif method == "rdma_infiniband_200":
        # 200Gb InfiniBand = ~25 GB/s, near-wire speed with RDMA
        bandwidth_gbps = 24
        latency_overhead_ms = 0.005  # ~5μs
        cpu_overhead = "Near zero"
    elif method == "rdma_infiniband_400":
        # 400Gb InfiniBand (NDR) = ~50 GB/s
        bandwidth_gbps = 48
        latency_overhead_ms = 0.003
        cpu_overhead = "Near zero"
    else:
        raise ValueError(f"Unknown method: {method}")
    
    transfer_time_ms = (data_size_gb / bandwidth_gbps) * 1000 + latency_overhead_ms
    
    return {
        "method": method,
        "bandwidth_gbps": bandwidth_gbps,
        "transfer_time_ms": transfer_time_ms,
        "cpu_overhead": cpu_overhead
    }

# KV cache sizes for different scenarios
kv_cache_sizes = [
    ("Llama-8B, 2K context", 0.5),
    ("Llama-8B, 8K context", 2.0),
    ("Llama-70B, 4K context", 10.0),
    ("Llama-70B, 32K context", 80.0),
]

methods = ["tcp_100gbe", "rdma_infiniband_200", "rdma_infiniband_400"]

print("KV Cache Transfer Time Comparison")
print("=" * 90)
print(f"{'Scenario':<30} {'Size':>8} {'TCP 100GbE':>15} {'IB 200Gb':>15} {'IB 400Gb':>15}")
print("-" * 90)

for scenario, size_gb in kv_cache_sizes:
    times = []
    for method in methods:
        result = estimate_transfer_time(size_gb, method)
        times.append(result['transfer_time_ms'])
    
    print(f"{scenario:<30} {size_gb:>6.1f}GB {times[0]:>12.1f}ms {times[1]:>12.1f}ms {times[2]:>12.1f}ms")

print("\nKey Insight: For large KV caches (70B+ models), RDMA is essential for disaggregated serving.")
print("Without it, transfer time dominates TTFT.")

---

## Part 5: Request Flow - Putting It All Together

Let's trace a complete request through the Dynamo system.

In [ ]:
# Complete request flow visualization

request_flow = """
REQUEST FLOW THROUGH DYNAMO (Disaggregated Mode)
================================================

1. CLIENT REQUEST
   ┌─────────────────────────────────────────────────────────────────────┐
   │ curl -X POST http://frontend:8000/v1/chat/completions \             │
   │   -d '{"model": "llama-3-8b", "messages": [{...}]}'                │
   └──────────────────────────────────┬──────────────────────────────────┘
                                      │
                                      ▼
2. FRONTEND (Rust HTTP Server - Port 8000)
   ┌─────────────────────────────────────────────────────────────────────┐
   │ • Validate OpenAI-compatible request                                │
   │ • Extract prompt, compute prefix hash                               │
   │ • Query Router: "Which worker for prefix hash abc123?"              │
   └──────────────────────────────────┬──────────────────────────────────┘
                                      │
                                      ▼
3. ROUTER (Rust, embedded in Frontend)
   ┌─────────────────────────────────────────────────────────────────────┐
   │ • Lookup prefix hash in global radix tree                           │
   │   - Query etcd: healthy workers                                     │
   │   - Query KV Block Manager: cache locations                         │
   │ • Decision: Prefill-Worker-2 has 78% cache hit, queue depth 3       │
   │ • Fallback: If no cache hit, pick least loaded prefill worker       │
   └──────────────────────────────────┬──────────────────────────────────┘
                                      │
                                      ▼
4. PREFILL WORKER (Python + vLLM/SGLang)
   ┌─────────────────────────────────────────────────────────────────────┐
   │ • Receive request via gRPC                                          │
   │ • Check local KV cache for prefix                                   │
   │   - HIT: Reuse cached KV blocks                                     │
   │   - MISS: Compute attention for all input tokens                    │
   │ • Publish to NATS: "dynamo.kv.cached" with block locations          │
   │ • Initiate NIXL transfer to Decode Worker                           │
   └──────────────────────────────────┬──────────────────────────────────┘
                                      │
                          ┌───────────┴───────────┐
                          │    NIXL Transfer      │
                          │  (RDMA, ~5μs/block)   │
                          └───────────┬───────────┘
                                      │
                                      ▼
5. DECODE WORKER (Python + vLLM/SGLang)
   ┌─────────────────────────────────────────────────────────────────────┐
   │ • Receive KV cache blocks in GPU memory                             │
   │ • Begin autoregressive token generation                             │
   │ • Stream tokens back through Frontend                               │
   │   - Token 1: "The" → 10ms                                          │
   │   - Token 2: "answer" → 10ms                                        │
   │   - Token 3: "is" → 10ms                                           │
   │   - ...                                                             │
   │ • On completion, publish metrics to NATS                            │
   └──────────────────────────────────┬──────────────────────────────────┘
                                      │
                                      ▼
6. RESPONSE TO CLIENT
   ┌─────────────────────────────────────────────────────────────────────┐
   │ HTTP/1.1 200 OK                                                     │
   │ Content-Type: text/event-stream                                     │
   │                                                                     │
   │ data: {"choices": [{"delta": {"content": "The"}}]}                 │
   │ data: {"choices": [{"delta": {"content": " answer"}}]}             │
   │ data: {"choices": [{"delta": {"content": " is"}}]}                 │
   │ ...                                                                 │
   │ data: [DONE]                                                        │
   └─────────────────────────────────────────────────────────────────────┘

BACKGROUND: PLANNER
   ┌─────────────────────────────────────────────────────────────────────┐
   │ • Subscribe to NATS: dynamo.planner.metrics                         │
   │ • Monitor: queue depths, TTFT, ITL, GPU utilization                 │
   │ • Detect: Long prompts increasing → prefill queue growing           │
   │ • Action: Scale prefill workers 2 → 4 (zero-downtime)               │
   │ • Update: etcd with new worker registrations                        │
   └─────────────────────────────────────────────────────────────────────┘
"""

print(request_flow)

---

## Part 6: Troubleshooting Guide

When things go wrong (and they will), check in this order:

In [ ]:
# Troubleshooting reference

troubleshooting_guide = {
    "Worker not found": {
        "likely_cause": "etcd registration failed or lease expired",
        "debug_commands": [
            "etcdctl get --prefix /dynamo/workers/",
            "etcdctl lease list",
            "curl http://frontend:8000/health"
        ],
        "fix": "Check worker logs for registration errors, verify etcd connectivity"
    },
    "Slow TTFT (Time-to-First-Token)": {
        "likely_cause": "KV cache miss - request routed to worker without cached prefix",
        "debug_commands": [
            "nats sub 'dynamo.kv.*'",
            "Check router logs for routing decisions",
            "nvidia-smi dmon -s u  # GPU utilization"
        ],
        "fix": "Verify KV-aware routing is enabled, check radix tree updates"
    },
    "Connection refused": {
        "likely_cause": "Service not started or wrong port",
        "debug_commands": [
            "ss -tlnp | grep 8000  # Frontend",
            "ss -tlnp | grep 2379  # etcd",
            "ss -tlnp | grep 4222  # NATS"
        ],
        "fix": "Start the service, check firewall rules"
    },
    "CUDA Out of Memory": {
        "likely_cause": "KV cache exhausted GPU memory",
        "debug_commands": [
            "nvidia-smi",
            "Check max_num_seqs and max_model_len settings",
            "nats sub 'dynamo.kv.evicted'  # Watch for evictions"
        ],
        "fix": "Reduce batch size, enable KV cache offloading to CPU"
    },
    "RDMA/NIXL transfer failed": {
        "likely_cause": "InfiniBand misconfiguration",
        "debug_commands": [
            "ibstat  # Check IB port state",
            "ibv_devinfo  # List RDMA devices",
            "ib_write_bw  # Test RDMA bandwidth"
        ],
        "fix": "Verify IB cables, check subnet manager, ensure GPUDirect is enabled"
    }
}

print("Dynamo Troubleshooting Quick Reference")
print("=" * 80)

for symptom, details in troubleshooting_guide.items():
    print(f"\n🔴 Symptom: {symptom}")
    print(f"   Likely cause: {details['likely_cause']}")
    print(f"   Debug commands:")
    for cmd in details['debug_commands']:
        print(f"      $ {cmd}")
    print(f"   Fix: {details['fix']}")
    print("-" * 80)

---

## Exercises

Complete these exercises to reinforce your understanding:

### Exercise 1: Calculate KV Cache Requirements

Your DGX Spark has 96GB GPU memory. You want to serve **Llama-3-70B** with up to **16K context length**.

1. How much memory does the model weights take (FP16)?
2. How much memory does a single 16K context KV cache take?
3. How many concurrent requests can you handle?

In [ ]:
# Exercise 1: Your solution here

# Hint: Llama-3-70B has 70 billion parameters
# FP16 = 2 bytes per parameter
# Use the KV cache formula from earlier

model_params_billions = 70
bytes_per_param = 2  # FP16
gpu_memory_gb = 96

# Model weights size
model_size_gb = ???  # TODO: Calculate

# KV cache per request (use calculate_kv_cache_size from earlier)
# Llama-3-70B: 80 layers, 8 KV heads, 128 head_dim
kv_cache_per_request_gb = ???  # TODO: Calculate

# Available memory for KV cache
available_for_kv_gb = ???  # TODO: Calculate

# Max concurrent requests
max_concurrent = ???  # TODO: Calculate

print(f"Model weights: {model_size_gb:.1f} GB")
print(f"KV cache per 16K request: {kv_cache_per_request_gb:.1f} GB")
print(f"Available for KV cache: {available_for_kv_gb:.1f} GB")
print(f"Max concurrent 16K requests: {max_concurrent}")

### Exercise 2: Design a Routing Strategy

You have 4 workers. Incoming request has prefix hash `abc123`. The KV Block Manager reports:

| Worker | Has Prefix `abc123`? | Queue Depth | GPU Util |
|--------|---------------------|-------------|----------|
| W1     | Yes (80% match)     | 5           | 85%      |
| W2     | No                  | 2           | 40%      |
| W3     | Yes (40% match)     | 8           | 90%      |
| W4     | No                  | 1           | 20%      |

Which worker should the router select? Consider:
1. Pure random routing - what would happen?
2. Least-loaded routing - which worker?
3. KV-aware routing - which worker and why?

In [ ]:
# Exercise 2: Your analysis here

workers = {
    "W1": {"cache_hit": 0.80, "queue_depth": 5, "gpu_util": 0.85},
    "W2": {"cache_hit": 0.00, "queue_depth": 2, "gpu_util": 0.40},
    "W3": {"cache_hit": 0.40, "queue_depth": 8, "gpu_util": 0.90},
    "W4": {"cache_hit": 0.00, "queue_depth": 1, "gpu_util": 0.20},
}

# TODO: Implement your routing decision
# Consider: cache hit rate saves compute, but queue depth adds latency

def route_request(workers: dict) -> str:
    """
    Return the worker ID that should handle this request.
    
    Hint: Balance cache hit benefit vs queue wait time.
    If 80% cache hit saves 800 tokens of prefill (400ms),
    is it worth waiting in a queue of 5?
    """
    # Your implementation here
    pass

selected = route_request(workers)
print(f"Selected worker: {selected}")
print(f"Reasoning: ???")  # TODO: Explain your choice

### Exercise 3: Trace a Multi-Turn Conversation

A user has a 3-turn conversation:

1. "What is Python?"
2. "Show me an example"
3. "Now in JavaScript"

For each turn, trace:
- Which component handles it
- Whether KV cache is reused
- What events are published to NATS

In [ ]:
# Exercise 3: Your trace here

conversation = [
    {"turn": 1, "user": "What is Python?"},
    {"turn": 2, "user": "Show me an example"},
    {"turn": 3, "user": "Now in JavaScript"},
]

# TODO: For each turn, document:
# - Cumulative prompt (includes previous turns)
# - Prefix hash (would be computed by router)
# - KV cache hit/miss
# - Worker selected
# - NATS events published

for turn in conversation:
    print(f"\nTurn {turn['turn']}: \"{turn['user']}\"")
    print(f"  Cumulative prompt: ???")
    print(f"  KV cache status: ???") 
    print(f"  Routed to: ???")
    print(f"  NATS events: ???")

---

## Key Takeaways

1. **Dynamo is distributed systems for LLM inference** - All patterns you know (service discovery, load balancing, caching, message queues) apply here.

2. **Two-phase inference is the key insight** - Prefill is compute-bound, decode is memory-bound. Disaggregating them enables specialization.

3. **KV cache is the new session state** - Routing to maximize cache hits is like session affinity but for computed attention states.

4. **etcd for "who exists", NATS for "what's happening"** - Service discovery vs event streaming serve different purposes.

5. **RDMA matters at scale** - When transferring gigabytes of KV cache between GPUs, kernel bypass is essential.

6. **Everything is observable** - Debug with etcdctl, nats-cli, nvidia-smi before diving into code.

---

## Next Steps

Now that you understand the architecture conceptually, proceed to:

**[Module 01: Setup and First Inference](01-setup-and-first-inference.ipynb)**

In the next module, we'll:
1. Install Dynamo on your DGX Spark
2. Start etcd and NATS manually
3. Launch a single worker with vLLM
4. Send your first inference request
5. Trace the request through the system with logs